In [ ]:
!pip install kaggle

In [ ]:
!pip install rouge

In [ ]:
import pandas as pd
import numpy as np
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail
License(s): CC0-1.0
 97% 487M/503M [00:03<00:00, 203MB/s]
100% 503M/503M [00:03<00:00, 150MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/newspaper-text-summarization-cnn-dailymail.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
df_train=pd.read_csv('/content/cnn_dailymail/train.csv')
df_test=pd.read_csv('/content/cnn_dailymail/test.csv')
df_val=pd.read_csv('/content/cnn_dailymail/validation.csv')

In [ ]:
sample_text = df_train['article'][3]
sample_summary = df_train['highlights'][3]
sample_text

"(CNN) -- With a breezy sweep of his pen President Vladimir Putin wrote a new chapter into Crimea's turbulent history, committing the region to a future returned to Russian domain. Sixty years prior, Ukraine's breakaway peninsula was signed away just as swiftly by Soviet leader Nikita Khrushchev. But dealing with such a blatant land grab on its eastern flank won't be anywhere near as quick and easy for Europe's 28-member union. Because, unlike Crimea's rushed referendum, everyone has a say. After initially slapping visa restrictions and asset freezes on a limited number of little known politicians and military men, Europe is facing urgent calls to widen the scope of its measures to target the Russian business community in particular. The logic of this is that those who run Russia and own it are essentially two sides of the coin. Alexei Navalny, one-time Moscow mayoral contender now under house arrest for opposing the current regime, called for Europe's leaders to ban everyone -- from V

In [ ]:
sample_summary

"Nina dos Santos says Europe must be ready to accept sanctions will hurt both sides .\nTargeting Russia's business community would be one way of sapping their support for President Putin, she says .\nBut she says Europe would have a hard time keeping its factories going without power from the east ."

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model and tokenizer
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def summarize_t5(text):
    inputs = tokenizer("summarize: " + text, max_length=1024, truncation=True, return_tensors="pt")
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=200, min_length=50, length_penalty=2.0, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
t5_summary = summarize_t5(sample_text)
t5_summary

"eric liu: a blatant land grab on Crimea's eastern flank won't be anywhere near as quick and easy. liu: the logic of this is that those who run Russia and own it are essentially two sides of the coin. liu: europe must be ready to accept sanctions are a two-way street and will hurt both sides."

In [ ]:
scores = rouge.get_scores([sample_summary], [t5_summary], avg=True)

# Print the ROUGE scores
print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")

ROUGE-1: 0.2889
ROUGE-2: 0.1569
ROUGE-L: 0.2889


In [ ]:
summaries_of_t5 = []
references_for_t5= []

for article in df_train['article'][:1000]:
    summary = summarize_t5(article)
    summaries_of_t5.append(summary)

for highlight in df_train['highlights'][:1000]:
    references_for_t5.append(highlight)

scores = rouge.get_scores(summaries_of_t5, references_for_t5, avg=True)

print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")

ROUGE-1: 0.3055
ROUGE-2: 0.1191
ROUGE-L: 0.2868
